In [1]:
print('This is 20250426/135610 version')

This is 20250426/135610 version


In [2]:

# %%
!pip install -q h5py typing-extensions wheel
!pip install -q -U bitsandbytes
# !pip install -q -U git+https://github.com/huggingface/transformers.git
# !pip install -q -U git+https://github.com/huggingface/peft.git
# !pip install -q -U git+https://github.com/huggingface/accelerate.git
# !pip install -q datasets

# %%
!nvidia-smi

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 24.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 28.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 70.4 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pylibcugraph-cu12 24.12.0 requires pylibraft-cu12==24.12.*, but you have pylibraft-cu12 25.2.0 which is incompatible.
pylibcugraph-cu12 24.12.0 requires rmm-cu12==24.12.*, but you have rm

In [3]:
# ## Load Pre-trained model and tokenizer

# %%
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
model_id = "Qwen/Qwen2.5-3B-Instruct"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True, # Activate nested quantization for 4-bit base models (double quantization)
    bnb_4bit_quant_type="nf4", # Quantization type (fp4 or nf4), According to QLoRA paper, for training 4-bit base models (e.g. using LoRA adapters) one should use
    bnb_4bit_compute_dtype=torch.bfloat16
)
# model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=bnb_config, device_map={"":0})
model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=bnb_config, device_map="auto")
tokenizer = AutoTokenizer.from_pretrained(model_id)


# %% [markdown]
# ## Preprocess the quantized model for training

# %%
from peft import prepare_model_for_kbit_training

model.gradient_checkpointing_enable()
model = prepare_model_for_kbit_training(model)

# %%
from peft import LoraConfig, get_peft_model

# You can try differnt parameter-effient strategy for model trianing, for more info, please check https://github.com/huggingface/peft
config = LoraConfig(
    r=8,
    lora_alpha=8,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
)

model = get_peft_model(model, config)
print("cell finished")

config.json:   0%|          | 0.00/661 [00:00<?, ?B/s]

2025-04-26 05:58:38.098122: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1745647118.317071      19 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1745647118.376712      19 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


model.safetensors.index.json:   0%|          | 0.00/35.6k [00:00<?, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/2.20G [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/7.30k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

cell finished


In [4]:
# %% [markdown]
# ## Chat Template Usage

# %%
from jinja2 import Template
template = Template(tokenizer.chat_template)
message = "Please introduce yourself"
print(f"message:\n{message}\n")
message_send_to_model=template.render(messages=[{"role": "user", "content": message}],bos_token=tokenizer.bos_token,add_generation_prompt=True)
print(f"message_send_to_model:\n{message_send_to_model}")

# %%
template = Template(tokenizer.chat_template)

@torch.no_grad()
def generate(prompt):
    modelInput=template.render(messages=[{"role": "user", "content": prompt}],bos_token= tokenizer.bos_token,add_generation_prompt=True)
    print("-"*80)
    print(f"model_input_string:\n{modelInput}")
    input_ids = tokenizer.encode(modelInput, add_special_tokens=False, return_tensors='pt').to("cuda:0")
    outputs = model.generate(input_ids, do_sample=False, max_new_tokens = 512)
    model_return_string = tokenizer.decode(*outputs, skip_special_tokens=False)
    print("-"*80)
    print(f"model_return_string:\n{model_return_string}")
    generated_ids = outputs[:, input_ids.shape[1]:]
    generated_text = tokenizer.decode(generated_ids[0], skip_special_tokens=False)
    return generated_text

query = "我被打了，有哪些途径可以维护自己的权益？"
print("-"*80)
print(f"query:\n{query}")
response = generate(query)
print("-"*80)
print(f"response:\n{response}")

# %% [markdown]
# ## Data Preparation

# %% [markdown]
# Let's load a common dataset, english quotes, to fine tune our model on famous quotes.

/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:631: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:636: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.8` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:653: UserWarning: `do_sample` is set to `False`. However, `top_k` is set to `20` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_k`.
  warnings.warn(
The attention mask is not set and cannot be inferred from input because pad token is same as eos toke

message:
Please introduce yourself

message_send_to_model:
<|im_start|>system
You are Qwen, created by Alibaba Cloud. You are a helpful assistant.<|im_end|>
<|im_start|>user
Please introduce yourself<|im_end|>
<|im_start|>assistant

--------------------------------------------------------------------------------
query:
我被打了，有哪些途径可以维护自己的权益？
--------------------------------------------------------------------------------
model_input_string:
<|im_start|>system
You are Qwen, created by Alibaba Cloud. You are a helpful assistant.<|im_end|>
<|im_start|>user
我被打了，有哪些途径可以维护自己的权益？<|im_end|>
<|im_start|>assistant

--------------------------------------------------------------------------------
model_return_string:
<|im_start|>system
You are Qwen, created by Alibaba Cloud. You are a helpful assistant.<|im_end|>
<|im_start|>user
我被打了，有哪些途径可以维护自己的权益？<|im_end|>
<|im_start|>assistant
如果您的权益受到了侵害，比如您被打了，您可以采取以下几种途径来维护自己的权益：

1. **报警**：如果您认为自己受到了身体上的伤害，或者对方的行为已经构成了违法行为（如故意伤害），那么最直接的方式就是报警。警方会进行调查，并根据情况

In [5]:
# original huatuo dataset
# # %%
# from datasets import load_dataset

# data = load_dataset("Abirate/english_quotes")

# dataset = load_dataset("FreedomIntelligence/Huatuo26M-Lite")
# dataset = dataset['train'].map(lambda sample: {"conversations": [{"from": "human", "value": sample['question']}, {"from": "gpt", "value": sample['answer']}]}, batched=False)

# dataset[3]

In [6]:
# load 法律数据集
from datasets import load_dataset, Features, Value, Sequence, concatenate_datasets
from datasets import DatasetDict
# 定义column转换方法
def convert_to_conversation(sample):
    return {
        "conversations": [
            {"from": "human", "value":  sample['input']},
            {"from": "gpt", "value": sample['output']}
        ]
    }

dataset = load_dataset("ShengbinYue/DISC-Law-SFT", data_files=['DISC-Law-SFT-Pair-QA-released.jsonl', 'DISC-Law-SFT-Pair.jsonl'])
print('cell finish')

README.md:   0%|          | 0.00/3.65k [00:00<?, ?B/s]

DISC-Law-SFT-Pair-QA-released.jsonl:   0%|          | 0.00/94.0M [00:00<?, ?B/s]

DISC-Law-SFT-Pair.jsonl:   0%|          | 0.00/347M [00:00<?, ?B/s]

Generating train split: 0 examples [00:00, ? examples/s]

cell finish


In [7]:
dataset_p1 = dataset
id_cat = set([id.rstrip('0123456789') for id in dataset_p1['train'].unique('id')])

# dataset_p1
desired_ids = {pre+'1' for pre in id_cat}
filtered_train = dataset_p1["train"].filter(lambda example: example["id"] in desired_ids)

# 更新 DatasetDict

filtered_dataset_1 = DatasetDict({"train": filtered_train})
print('cell finished')

Filter:   0%|          | 0/246450 [00:00<?, ? examples/s]

cell finished


In [8]:
# 10 question for model evaluation
print(id_cat)
filtered_dataset_1['train']

{'jud_read_compre-', 'leg_ele_extra-', 'sim_case_match-', 'jud_doc_sum-', 'leg_case_cls-', 'exam-', 'sent_pred-', 'legal_question_answering_', 'op_sum-', 'leg_eve_detec-'}


Dataset({
    features: ['id', 'input', 'output'],
    num_rows: 10
})

In [9]:
# 应用转换
dataset = dataset['train'].map(
    convert_to_conversation,
    remove_columns=dataset['train'].column_names,  # 移除原始列
    batched=False
)

features = Features({
    'id': Value('string'),
    'reference': Sequence(Value('string')),
    'input': Value('string'),
    'output': Value('string')
})

dataset2 = load_dataset("ShengbinYue/DISC-Law-SFT", features = features,data_files=['DISC-Law-SFT-Triplet-QA-released.jsonl', 'DISC-Law-SFT-Triplet-released.jsonl'])
print('cell finish')

Map:   0%|          | 0/246450 [00:00<?, ? examples/s]

DISC-Law-SFT-Triplet-QA-released.jsonl:   0%|          | 0.00/85.5M [00:00<?, ?B/s]

DISC-Law-SFT-Triplet-released.jsonl:   0%|          | 0.00/52.8M [00:00<?, ?B/s]

Generating train split: 0 examples [00:00, ? examples/s]

cell finish


In [10]:
dataset_p2 = dataset2

id_cat = set([id.rstrip('0123456789') for id in dataset_p2['train'].unique('id')])

# dataset_p1
desired_ids = desired_ids = {pre+'1' for pre in id_cat}
filtered_train = dataset_p2["train"].filter(lambda example: example["id"] in desired_ids)

# 更新 DatasetDict
print(id_cat)
filtered_dataset_2 = DatasetDict({"train": filtered_train})
filtered_dataset_2
print('cell finish')

Filter:   0%|          | 0/39331 [00:00<?, ? examples/s]

{'legal_question_answering_', 'judgement_predit-'}
cell finish


In [11]:
dataset2 = dataset2['train'].map(
    convert_to_conversation,
    remove_columns=dataset2['train'].column_names,  # 移除原始列
    batched=False
)

merged_dataset = concatenate_datasets([dataset, dataset2])
merged_dataset

Map:   0%|          | 0/39331 [00:00<?, ? examples/s]

Dataset({
    features: ['conversations'],
    num_rows: 285781
})

In [12]:
filtered_dataset_2['train'][1]

{'id': 'judgement_predit-1',
 'reference': ['《刑法》第一百一十四条：【放火罪】【决水罪】【爆炸罪】【投放危险物质罪】【以危险方法危害公共安全罪】放火、决水、爆炸以及投放毒害性、放射性、传染病病原体等物质或者以其他危险方法危害公共安全，尚未造成严重后果的，处三年以上十年以下有期徒刑。'],
 'input': '基于下列案件，推测可能的判决结果。\n经审理查明，2015年6月21日15时许，被告人白某某在大东区小河沿公交车站乘坐被害人张某某驾驶的133路公交车，当车辆行驶至沈阳市大东区东陵西路26号附近时，被告人白某某因未能下车而与司机张某某发生争执，并在该公交车行驶中用手拉拽档杆，被证人韩某某拉开后，被告人白某某又用手拉拽司机张某某的右胳膊，导致该车失控撞向右侧马路边停放的轿车和一个路灯杆，路灯杆折断后将福锅记炖品店的牌匾砸坏。后经被害人张某某报警，公安人员赶至现场将被告人白某某传唤到案。经鉴定，公交车受损价值人民币5,189.9元，轿车受损价值人民币1,449.57元，路灯杆受损价值人民币2,927.15元，福锅记饭店牌匾受损价值人民币9,776元，本案损失价值共计人民币19,342.6元。上述事实，被告人白某某在庭审中亦无异议，被害人张某某、朱某某、詹某某陈述，证人韩某某的证言，现场勘察笔录，视听资料，鉴定结论书，被告人白某某的供述与辩解等证据证实，足以认定。',
 'output': '根据《刑法》第一百一十四条的规定，被告人白某某以危险方法危害公共安全，尚未造成严重后果。根据案件事实，被告人白某某在公交车上与司机发生争执，并用手拉拽档杆和司机的胳膊，导致公交车失控撞向停放的轿车和路灯杆，最终造成公交车、轿车、路灯杆以及福锅记饭店牌匾的损坏。根据鉴定结果，本案损失共计人民币19,342.6元。\n综上所述，根据《刑法》第一百一十四条的规定，被告人白某某的行为构成以危险方法危害公共安全罪。根据该罪的刑罚幅度，可判处三年以上十年以下有期徒刑。\n需要注意的是，以上仅为根据提供的案件事实和相关法律条文进行的推理，最终的判决结果还需根据法庭审理的具体情况和法官的判断做出。'}

In [13]:
# dataset_1_prefinetuned = [{'id': data['id'], 'input': data['input'], 'output': data['output'], 'model': generate(data['input'])} for data in filtered_dataset_1['train']]

In [14]:
# dataset_1_prefinetuned[0]['model']

In [15]:
# dataset_2_prefinetuned = [{'id': data['id'], 'input': data['input'], 'output': data['output'], 'model': generate(data['input'])} for data in filtered_dataset_2['train']]

In [16]:
# import json

# output_file = "dataset_1_prefinetuned.json"

# # 写入文件（包含格式化与编码处理）
# with open(output_file, 'w', encoding='utf-8') as f:
#     json.dump(
#         dataset_1_prefinetuned,
#         f,
#         ensure_ascii=False,  # 允许保存非ASCII字符（如中文）
#         indent=4            # 缩进美化，便于阅读
#     )

In [17]:
# output_file = "dataset_2_prefinetuned.json"

# # 写入文件（包含格式化与编码处理）
# with open(output_file, 'w', encoding='utf-8') as f:
#     json.dump(
#         dataset_2_prefinetuned,
#         f,
#         ensure_ascii=False,  # 允许保存非ASCII字符（如中文）
#         indent=4            # 缩进美化，便于阅读
#     )

In [18]:
# import torch
# from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
# model_id = "Qwen/Qwen2.5-3B-Instruct"
# tokenizer = AutoTokenizer.from_pretrained(model_id)

In [19]:
# %%
from torch.utils.data import random_split
# train_dataset_size, val_dataset_size = 40, 8
# train_dataset, val_dataset, _ = random_split(dataset, [train_dataset_size, val_dataset_size, len(dataset)-train_dataset_size-val_dataset_size])
train_dataset, val_dataset, _ = random_split(dataset2, [0.2,0.01,0.79])
# print(train_dataset[0]['conversations'])

# %% [markdown]
# ### Customized Dataset
# Create a specialized dataset class named "InstructionDataset" designed to handle our custom dataset.

# %%
import transformers
from typing import Dict, Sequence, List
from torch.utils.data import Dataset
from dataclasses import dataclass

def preprocess(
    sources,
    tokenizer: transformers.PreTrainedTokenizer,
) -> Dict:
    template = Template(tokenizer.chat_template)
    max_seq_len = tokenizer.model_max_length
    messages = []
    for i, source in enumerate(sources):
        if source[0]["from"] != "human":
            # Skip the first one if it is not from human
            source = source[1:]

        for j in range(0, len(source), 2):
            if j+1 >= len(source): continue
            q = source[j]["value"]
            a = source[j+1]["value"]
            assert q is not None and a is not None, f'q:{q} a:{a}'
            input =  template.render(messages=[{"role": "user", "content": q},{"role": "assistant", "content": a}],bos_token=tokenizer.bos_token,add_generation_prompt=False)
            input_ids = tokenizer.encode(input, add_special_tokens= False)

            query = template.render(messages=[{"role": "user", "content": q}],bos_token=tokenizer.bos_token,add_generation_prompt=True)
            query_ids = tokenizer.encode(query, add_special_tokens= False)

            labels = [-100]*len(query_ids) + input_ids[len(query_ids):]
            assert len(labels) == len(input_ids)
            if len(input_ids) == 0: continue
            messages.append({"input_ids": input_ids[-max_seq_len:], "labels": labels[-max_seq_len:]})

    input_ids = [item["input_ids"] for item in messages]
    labels = [item["labels"] for item in messages]

    max_len = max(len(x) for x in input_ids)

    max_len = min(max_len, max_seq_len)
    input_ids = [ item[:max_len] + [tokenizer.eos_token_id]*(max_len-len(item)) for item in input_ids]
    labels = [ item[:max_len] + [-100]*(max_len-len(item)) for item in labels]

    input_ids = torch.LongTensor(input_ids)
    labels = torch.LongTensor(labels)
    return {
        "input_ids": input_ids,
        "labels": labels
    }


class InstructDataset(Dataset):
    def __init__(self, data: Sequence, tokenizer: transformers.PreTrainedTokenizer) -> None:
        super().__init__()
        self.tokenizer = tokenizer
        self.data = data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index) -> Dict[str, torch.Tensor]:
        sources = self.data[index]
        if isinstance(index, int):
            sources = [sources]
        data_dict = preprocess([e['conversations'] for e in sources], self.tokenizer)
        if isinstance(index, int):
            data_dict = dict(input_ids=data_dict["input_ids"][0], labels=data_dict["labels"][0])
        return data_dict


@dataclass
class DataCollatorForSupervisedDataset(object):
    tokenizer: transformers.PreTrainedTokenizer
    def __call__(self, instances: Sequence[Dict]) -> Dict[str, torch.Tensor]:
        input_ids, labels = tuple([instance[key] for instance in instances] for key in ("input_ids", "labels"))
        input_ids = torch.nn.utils.rnn.pad_sequence(
            input_ids,
            batch_first=True,
            padding_value=self.tokenizer.pad_token_id)
        labels = torch.nn.utils.rnn.pad_sequence(labels, batch_first=True, padding_value=IGNORE_INDEX)
        return dict(
            input_ids=input_ids,
            labels=labels,
            attention_mask=input_ids.ne(self.tokenizer.pad_token_id),
        )

# %%
train_dataset = InstructDataset(train_dataset, tokenizer)
val_dataset = InstructDataset(val_dataset, tokenizer)
data_collator = DataCollatorForSupervisedDataset(tokenizer=tokenizer)
print("cell finished")

cell finished


In [20]:
!nvidia-smi

Sat Apr 26 06:00:19 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 560.35.03              Driver Version: 560.35.03      CUDA Version: 12.6     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   47C    P0             28W /   70W |    2451MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [21]:
# %%
print('='*60)
print('training cell start')
print('='*60)
sample_data = train_dataset[9]
IGNORE_INDEX=-100

print("=" * 80)
print("Debuging: ")
print(f"Input_ids\n{sample_data['input_ids']}")
print(f"Label_ids\n{sample_data['labels']}")
print("-" * 80)
print(f"Input:\n{tokenizer.decode(sample_data['input_ids'])}")
print("-" * 80)
N_id = tokenizer.encode("N", add_special_tokens= False)[0]
print(f"Label:\n{tokenizer.decode([N_id if x == -100 else x for x in sample_data['labels']])}")
print("=" * 80)


# %% [markdown]
# ## Training

# %% [markdown]
# ### General Training Hyperparameters

# %%
# Set training parameters
training_arguments = transformers.TrainingArguments(
    output_dir="./checkpoints" ,
    # resume_from_checkpoint = "./checkpoints",
    num_train_epochs=1,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=2,
    gradient_accumulation_steps=2,
    optim='paged_adamw_32bit',
    save_steps=0,
    logging_steps=1,
    learning_rate=2e-7,
    weight_decay=0.001,
    max_steps=-1,
    warmup_ratio=0.03,
    group_by_length=True,
    lr_scheduler_type="cosine",
    gradient_checkpointing=True,
    report_to="none",
    dataloader_drop_last=True
)

# %%

model.train()
trainer = transformers.Trainer(
    model=model,
    tokenizer=tokenizer,
    args=training_arguments,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    data_collator=data_collator
)
trainer.train()
print('finish training')

training cell start
Debuging: 
Input_ids
tensor([151644,   8948,    198,   2610,    525,   1207,  16948,     11,   3465,
           553,  54364,  14817,     13,   1446,    525,    264,  10950,  17847,
            13, 151645,    198, 151644,    872,    198, 120492, 112634,  99532,
        103424, 114006,   3837,     17,     15,     16,     22,   7948,     22,
        104081,  99919,   3837, 108360, 104941,  99569,  18493, 103927,  17447,
         99850,  11622, 102279,  89047, 105701,   2073,  18830,  99477, 110789,
           854, 106140, 100010,   3837,  75882, 100010, 104548, 109536, 103965,
         61689, 102129,   3837,  69041, 102129, 103965,  99553,  62189,  81217,
         99107, 105208, 108673, 118313,  87140,     24,     17,  18947,   5373,
        108673, 118313,  45930,     17,     16,     17,  86341,   1773, 105379,
         70589,  87140,  81217,  45930,  99472, 100409, 108673, 118313, 104613,
          8997, 104210,  31838,  89012,   9370, 101995,  27369,   3837, 102086,

/tmp/ipykernel_19/2043322358.py:52: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = transformers.Trainer(
No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.


Step,Training Loss
1,0.907500
2,0.750300
3,0.862700
4,1.034200
5,0.815800
6,1.030000
7,0.788200
8,0.853400
9,0.904000
10,0.945100


finish training


In [22]:
# %%
def print_trainable_parameters(model):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}"
    )

model.print_trainable_parameters()

# %% [markdown]
# Once the training is completed, we can evaluate our model and get its perplexity on the validation set like this:

# %%
import math
# !pip install -q -U git+https://github.com/huggingface/accelerate.git
eval_results = trainer.evaluate()
print(f"Perplexity: {math.exp(eval_results['eval_loss']):.2f}")


# %% [markdown]
# ## Save Trained LoRA

trainable params: 1,843,200 || all params: 3,087,781,888 || trainable%: 0.0597


Perplexity: 2.68


In [23]:
# %%
!pwd
output_path = "ilora"
trainer.save_model(output_path)

# %% [markdown]
# ### Test the trained model

# %%
template = Template(tokenizer.chat_template)
@torch.no_grad()
def generate(prompt):
    modelInput = template.render(messages=[{"role": "user", "content": prompt}],bos_token= tokenizer.bos_token,add_generation_prompt=True)
    input_ids = tokenizer.encode(modelInput, add_special_tokens=False, return_tensors='pt').to("cuda:0")
    outputs = model.generate(input_ids, temperature=1.0, max_new_tokens = 500)
    model_return_string = tokenizer.decode(*outputs, skip_special_tokens=False)
    print("-"*80)
    print(f"model_return_string:\n{model_return_string}")
    generated_ids = outputs[:, input_ids.shape[1]:]
    generated_text = tokenizer.decode(generated_ids[0], skip_special_tokens=False)
    return generated_text

query = "我被打了，我可以寻求哪些法律援助？"
print(f"query:\n{query}")
response = generate(query)
print("-"*80)
print(f"response:\n{response}")

# %% [markdown]
# # Clean GPU Memory

# %%
!nvidia-smi

/kaggle/working


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


query:
我被打了，我可以寻求哪些法律援助？
--------------------------------------------------------------------------------
model_return_string:
<|im_start|>system
You are Qwen, created by Alibaba Cloud. You are a helpful assistant.<|im_end|>
<|im_start|>user
我被打了，我可以寻求哪些法律援助？<|im_end|>
<|im_start|>assistant
如果你遭受了身体或心理上的伤害，并且这种行为符合法律定义的犯罪（例如：暴力威胁、身体攻击等），你应该立即寻求法律帮助。在中国，你可以采取以下几种方式寻求法律援助：

1. **报警**：如果你认为自己的安全受到威胁，应立即报警。报警后，警方会根据情况进行调查并可能采取强制措施。

2. **寻求法律帮助**：
   - **找律师**：联系专业的律师或法援中心以获取法律建议和帮助。在中国，有些地方的法援中心可以免费为有需要的人提供法律服务。
   - **法律援助机构**：联系当地的法律援助中心或志愿者律师组织寻求帮助。

3. **申请保护令**：如果你认为自己处于危险之中，或者担心对方会再次施暴，你可以向法院申请保护令。这可以帮助保护你的安全和权益。

4. **心理健康支持**：遭受攻击后可能会经历情绪波动或其他心理健康问题，可以考虑寻求心理咨询师或精神科医生的帮助。

5. **教育和信息**：了解相关的法律法规，参加相关教育活动，提高自我保护意识和能力。

请记住，在任何情况下，你的安全是最重要的。如果遇到这种情况，请务必尽快采取行动保护自己。<|im_end|>
--------------------------------------------------------------------------------
response:
如果你遭受了身体或心理上的伤害，并且这种行为符合法律定义的犯罪（例如：暴力威胁、身体攻击等），你应该立即寻求法律帮助。在中国，你可以采取以下几种方式寻求法律援助：

1. **报警**：如果你认为自己的安全受到威胁，应立即报警。报警后，警方会根

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [24]:
# %%
# Empty VRAM
del model
del trainer
import gc
import torch
torch.cuda.empty_cache()
gc.collect()
gc.collect()



0

In [25]:
# %%
!nvidia-smi

Sat Apr 26 13:12:39 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 560.35.03              Driver Version: 560.35.03      CUDA Version: 12.6     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   65C    P0             40W /   70W |    2451MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [26]:
# %% [markdown]
# ## Load the trained model back and integrate the trained LoRA within.

# %%

from jinja2 import Template
template = Template(tokenizer.chat_template)
message = "Please introduce yourself"
print(f"message:\n{message}\n")
message_send_to_model=template.render(messages=[{"role": "user", "content": message}],bos_token=tokenizer.bos_token,add_generation_prompt=True)
print(f"message_send_to_model:\n{message_send_to_model}")

# %%
template = Template(tokenizer.chat_template)

from peft import PeftModel
output_path = "ilora"
IGNORE_INDEX=-100

# model = AutoModelForCausalLM.from_pretrained(model_id, load_in_8bit=True, device_map={"":0})
model = AutoModelForCausalLM.from_pretrained(model_id, load_in_8bit=True, device_map='auto')
model = PeftModel.from_pretrained(model, output_path, is_trainable = True)
model = model.merge_and_unload()
model.config.max_length = 512
# model.eval()

tokenizer = transformers.AutoTokenizer.from_pretrained(model_id, padding_side="left")
# tokenizer.pad_token = tokenizer.unk_token


# ======================== resume from checkpoint ==============================
from peft import prepare_model_for_kbit_training

model.gradient_checkpointing_enable()
model = prepare_model_for_kbit_training(model)

# %%
from peft import LoraConfig, get_peft_model

# You can try differnt parameter-effient strategy for model trianing, for more info, please check https://github.com/huggingface/peft
config = LoraConfig(
    r=8,
    lora_alpha=8,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
)

model = get_peft_model(model, config)
print("cell finished")


# training_arguments = transformers.TrainingArguments(
#     output_dir="./checkpoints_phase2" ,
#     resume_from_checkpoint = True,
#     num_train_epochs=1,
#     per_device_train_batch_size=2,
#     per_device_eval_batch_size=2,
#     gradient_accumulation_steps=2,
#     optim='paged_adamw_32bit',
#     save_steps=0,
#     logging_steps=1,
#     learning_rate=2e-7,
#     weight_decay=0.001,
#     max_steps=-1,
#     warmup_ratio=0.03,
#     group_by_length=True,
#     lr_scheduler_type="cosine",
#     gradient_checkpointing=True,
#     report_to="none",
#     dataloader_drop_last=True
# )

# # %%

# model.train()
# trainer = transformers.Trainer(
#     model=model,
#     tokenizer=tokenizer,
#     args=training_arguments,
#     train_dataset=train_dataset,
#     eval_dataset=val_dataset,
#     data_collator=data_collator
# )
# trainer.train()

message:
Please introduce yourself

message_send_to_model:
<|im_start|>system
You are Qwen, created by Alibaba Cloud. You are a helpful assistant.<|im_end|>
<|im_start|>user
Please introduce yourself<|im_end|>
<|im_start|>assistant



The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/usr/local/lib/python3.11/dist-packages/peft/tuners/lora/bnb.py:85: UserWarning: Merge lora module to 8-bit linear may get different generations due to rounding errors.
  warnings.warn(


cell finished


In [27]:
# # %% [markdown]
# # ## Answer generation

# # %%
# @torch.no_grad()
# def generate(prompts):
#     model_inputs = [template.render(messages=[{"role": "user", "content": prompt}], bos_token=tokenizer.bos_token, add_generation_prompt=True) for prompt in prompts]
#     input_ids = tokenizer(model_inputs, add_special_tokens=False, return_tensors='pt', padding=True).to("cuda:0")

#     outputs = model.generate(input_ids.input_ids,attention_mask=input_ids.attention_mask, max_new_tokens=500)

#     generated_texts = []
#     for i in range(len(prompts)):
#         generated_ids = outputs[i, input_ids.input_ids.shape[1]:]
#         generated_text = tokenizer.decode(generated_ids, skip_special_tokens=True, clean_up_tokenization_spaces=True)
#         generated_texts.append(generated_text)

#     return generated_texts

# # test
# print("\n\n".join(generate("和妻子发生离婚财产纠纷，属于什么案件？刑事还是民事？")))


# # %%
# # !wget https://NLP-course-cuhksz.github.io/Assignments/Assignment1/task1/data/1.exam.json

In [28]:
import json

# dataset_1_finetuned = [{'id': data['id'], 'input': data['input'], 'output': data['output'], 'model': generate(data['input'])} for data in filtered_dataset_1['train']]

# output_file = "dataset_1_finetuned.json"

# # 写入文件（包含格式化与编码处理）
# with open(output_file, 'w', encoding='utf-8') as f:
#     json.dump(
#         dataset_1_finetuned,
#         f,
#         ensure_ascii=False,  # 允许保存非ASCII字符（如中文）
#         indent=4            # 缩进美化，便于阅读
#     )
# print('cell finished')

In [29]:
dataset_2_finetuned = [{'id': data['id'], 'input': data['input'], 'output': data['output'], 'model': generate(data['input'])} for data in filtered_dataset_2['train']]

output_file = "dataset_2_finetuned.json"

# 写入文件（包含格式化与编码处理）
with open(output_file, 'w', encoding='utf-8') as f:
    json.dump(
        dataset_2_finetuned,
        f,
        ensure_ascii=False,  # 允许保存非ASCII字符（如中文）
        indent=4            # 缩进美化，便于阅读
    )

/usr/local/lib/python3.11/dist-packages/bitsandbytes/autograd/_functions.py:315: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")


--------------------------------------------------------------------------------
model_return_string:
<|im_start|>system
You are Qwen, created by Alibaba Cloud. You are a helpful assistant.<|im_end|>
<|im_start|>user
《民法典》第九百七十四条：除合伙合同另有约定外，合伙人向合伙人以外的人转让其全部或者部分财产份额的，须经其他合伙人一致同意。
《民法典》第九百七十五条：合伙人的债权人不得代位行使合伙人依照本章规定和合伙合同享有的权利，但是合伙人享有的利益分配请求权除外。
《民法典》第九百七十三条：合伙人对合伙债务承担连带责任。清偿合伙债务超过自己应当承担份额的合伙人，有权向其他合伙人追偿。
《民法典》第九百七十六条：合伙人对合伙期限没有约定或者约定不明确，依据本法第五百一十条的规定仍不能确定的，视为不定期合伙。合伙期限届满，合伙人继续执行合伙事务，其他合伙人没有提出异议的，原合伙合同继续有效，但是合伙期限为不定期。合伙人可以随时解除不定期合伙合同，但是应当在合理期限之前通知其他合伙人。
<问题>：
在不定期合伙的情况下，一个合伙人突然退出并清偿了其应当承担份额的合伙债务后，是否有权向其他合伙人追偿？<|im_end|>
<|im_start|>assistant
在不定期合伙关系中，根据《民法典》第九百七十六条的规定，如果合伙人对合伙期限没有约定或约定不明，并且依据该法条第510条规定仍无法确定合伙期限的话，这种合伙关系将被视为不定期合伙。在这种情况下，合伙人有权利随时解除合伙合同，但需在合理期限前通知其他合伙人。

关于您提到的具体情况，在不定期合伙关系下，如果一个合伙人退出时已经清偿了其应承担的合伙债务，并且在合理期限内通知了其他合伙人，那么该合伙人是否能够向其他合伙人追偿则要看具体情况。

具体来说，如果该合伙人按照合伙合同的约定以及法律规定，履行了告知义务并且未违反其他合伙人的权益，且合伙合同继续有效的前提下（即其他合伙人没有在合理期限内表示反对），该合伙人通常可以向其他合伙人追偿超过其应当承担份额的部分。

然而，需要注意的是，《民法典》第九百七

In [30]:
# # %%
# import json

# with open('1.exam.json') as f:
#   data = json.load(f)
#   data = data[:20] # just for demo

# print(data[0])

# # %%
# your_prompt = """请回答下面的多选题，你需要先给出一段解答分析，然后再按照下面的格式样例给出正确答案，格式样例：[答案] B
# {question}
# {options}"""

# def get_query(da):
#   da['options'] = '\n'.join([f"{k}:{v}" for k, v in da['option'].items() if v])
#   return your_prompt.format_map(da)

# for item in data:
#   item['query'] = get_query(item)


# print(data[0]['query'])

# # %%
# model_answers = generate([item['query'] for item in data])
# print(f'\n{model_answers[0]}')

# # %%
# import re
# from tqdm import tqdm

# def get_ans(ans):
#     match = re.findall(r'.*?([A-E]+(?:[、, ]+[A-E]+)*)', ans)
#     if match:
#         last_match = match[-1]
#         return ''.join(re.split(r'[、, ，]+', last_match))
#     return ''

# correct_num = 0
# total_num = 0
# for model_answer, item in tqdm(zip(model_answers, data)):
#   if get_ans(model_answer) == item['answer']:
#     correct_num += 1
#   total_num += 1
#   item['model_answer'] = model_answer

# print(f"ACC: {correct_num/total_num:.2%}")

# result_path = "/content/result.jso"
# with open(result_path, "w", encoding="utf-8") as file:
#     json.dump(data, file, ensure_ascii=False, indent=4)
#     print(f"Results are save in {result_path}")